# Data wrangling project - Dog rating data set from Twitter

In [1]:
# import all necessary libraries
import pandas as pd
import requests
import tweepy
import os
import json

# Gather data

## 1. The WeRateDogs Twitter archive from csv file

In [2]:
Dogarchive=pd.read_csv('twitter-archive-enhanced.csv')
Dogarchive.head()
Dogarchive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

## 2. Programatic download of tsv data with Dog image urls

In [3]:
url='https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response=requests.get(url)
folder_name='tweet_image'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
with open(os.path.join(folder_name,url.split('/')[-1]),mode='wb') as file:
    file.write(response.content)

In [6]:
# storing the tsv data into a dataframe
imagepred=pd.read_csv(os.path.join(folder_name,"image-predictions.tsv"),delimiter="\t")

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


## 2. Data from Twitter

In [7]:
consumer_key = 'qMvi7Z4iRPMGKYgkXrDVJdbL9'
consumer_secret = 'IXhEjTP8NXDkZzdAVjFSaPtRjZQdrtxaABuvMKO6hS4cAX7Jee'
access_token = '100096730-1QZhfMa4QD5060D5QVxiUH1bbUwu3isJJlL16izg'
access_secret = 'BGI9ulFajY8ej8Tm4cZP1yKluUDK62YbidNlBFaGxOmec'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [8]:
tweet_id=Dogarchive['tweet_id']
list_of_tweets=[]
no_tweets_found=[]
for tweet in tweet_id:
    try:
        list_of_tweets.append(api.get_status(tweet))
    except Exception as e:
        no_tweets_found.append(tweet)

In [9]:
print("Number of tweets identified ",len(list_of_tweets))
print("Number of tweets not identified ",len(no_tweets_found))

Number of tweets identified  500
Number of tweets not identified  1856


In [10]:
list_of_tweets[0]

Status(_api=<tweepy.api.API object at 0x000002499939A848>, _json={'created_at': 'Tue Aug 01 16:23:56 +0000 2017', 'id': 892420643555336193, 'id_str': '892420643555336193', 'text': "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU", 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 892420639486877696, 'id_str': '892420639486877696', 'indices': [86, 109], 'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'url': 'https://t.co/MgUWQ76dJU', 'display_url': 'pic.twitter.com/MgUWQ76dJU', 'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'medium': {'w': 540, 'h': 528, 'resize': 'fit'}, 'small': {'w': 540, 'h': 528, 'resize': 'fit'}, 'large': {'w': 540, 'h': 528, 'resize': 'fit'}}}]},

In [30]:
# store json data into text file
with open('tweet_json.txt',"w") as file:
    for tweet_data in list_of_tweets:
        data=json.dumps(tweet_data._json)
        file.write(data+"\n")

In [33]:
# Create a dataframe from the tweet data that we just retrieved.
tweet_data = []
with open('tweet_json.txt') as json_file:
    for json_obj in json_file:
        tweet_shortdata_dict = {}
        tweet_alldata_dict = json.loads(json_obj)

        # Get all of the data we're interested in
        tweet_shortdata_dict['tweet_id'] = tweet_alldata_dict['id']
        tweet_shortdata_dict['retweet_count'] = tweet_alldata_dict['retweet_count']
        tweet_shortdata_dict['favorite_count'] = tweet_alldata_dict['favorite_count']

        # Append it to the data gathering list
        tweet_data.append(tweet_shortdata_dict)

# Create a dataframe from the data
tweet_data_df = pd.DataFrame(tweet_data)
tweet_data_df.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7626,35909
1,892177421306343426,5644,30985
2,891815181378084864,3738,23312
3,891689557279858688,7793,39194
4,891327558926688256,8392,37442


# Assessing data

## Manual assessment

In [ ]:
Dogarchive.head()

In [ ]:
imagepred.head()